### Layer Generation

This file converts arcgis FeatureServer data into pmtiles and uploads them to a supabase bucket.

To add a new file: Add the arcgis URL path to the `LAYER_MAP` object.


In [8]:
LAYER_MAP = {
    # "path_lines": "https://services3.arcgis.com/Stu7jwuXrnM0myT0/arcgis/rest/services/PATH_Train/FeatureServer/0",
    # "path_stations": "https://services3.arcgis.com/Stu7jwuXrnM0myT0/arcgis/rest/services/PATH_Train/FeatureServer/0",
    # "nyc_lines": "https://services5.arcgis.com/OKgEWPlJhc3vFb8C/arcgis/rest/services/MTA_Subway_Routes_Stops/FeatureServer/1",
    # "nyc_stations": "https://services5.arcgis.com/OKgEWPlJhc3vFb8C/arcgis/rest/services/MTA_Subway_Routes_Stops/FeatureServer/0",
    # "nj_light_rail_lines": "https://services6.arcgis.com/M0t0HPE53pFK525U/arcgis/rest/services/NJTransit_Light_Rail/FeatureServer/0",
    # "nj_light_rail_stations": "https://services6.arcgis.com/M0t0HPE53pFK525U/arcgis/rest/services/NJTransit_Light_Rail_Stations/FeatureServer/0",
    # "nj_rail_lines": "https://services6.arcgis.com/M0t0HPE53pFK525U/arcgis/rest/services/NJTRANSIT_RAIL_LINES_1/FeatureServer/0",
    # "nj_rail_stations": "https://services6.arcgis.com/M0t0HPE53pFK525U/arcgis/rest/services/NJTransit_Rail_Stations/FeatureServer/0",
    "nyc_bike_lanes": "https://services9.arcgis.com/gbyGLR8owy6Q4kUA/arcgis/rest/services/NYC_Bike_Lane_Network/FeatureServer/0",
}

In [9]:
import os
from supabase import create_client
import dotenv
dotenv.load_dotenv()
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_ANON_KEY")
supabase = create_client(url, key)

def upload_to_supabase(file_path, bucket_name):
    # Read the file
    with open(file_path, "rb") as file:
        # Upload the file to the specified bucket
        response = supabase.storage.from_(bucket_name).upload(file_path, file)
        return response

# Example usage
# upload_to_supabase('./layer_outputs/nj_rail_stations.pmtiles', 'your_bucket_name')

In [10]:
import requests
import json
import subprocess
import os


def arcgis_url_to_pmtiles(name, url):
    query_url = f"{url}/query"

    params = {
        "where": "1=1",
        "outFields": "*",
        "f": "geojson",
        "returnGeometry": "true",
    }

    response = requests.get(query_url, params=params)
    geojson_data = response.json()

    # Save GeoJSON
    with open(f"./geo_layers/{name}.geojson", "w") as f:
        json.dump(geojson_data, f)

    # Convert to PMTiles using tippecanoe
    subprocess.run(
        [
            "tippecanoe",
            "-o",
            f"./geo_layers/{name}.pmtiles",
            "-zg",  # Auto-determine zoom levels
            "--drop-densest-as-needed",
            f"./geo_layers/{name}.geojson",
            "--force",
        ]
    )

In [11]:
#This will fail if the file already exists. Must delete first.

BUCKET_NAME = "citi-bike-data-bucket"
for layer_name, arcgis_url in LAYER_MAP.items():
    arcgis_url_to_pmtiles(layer_name, arcgis_url)
    file_path = f"./geo_layers/{layer_name}.pmtiles"
    upload_response = upload_to_supabase(file_path, BUCKET_NAME)

For layer 0, using name "nyc_bike_lanes"
2000 features, 210241 bytes of geometry and attributes, 70564 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
Choosing a maxzoom of -z9 for features typically 712 feet (217 meters) apart, and at least 165 feet (51 meters) apart
Choosing a maxzoom of -z10 for resolution of about 477 feet (145 meters) within features
  99.9%  10/301/385  


StorageApiError: {'statusCode': 409, 'error': Duplicate, 'message': The resource already exists}